# Testing visual document entity retrieval by existence of visual entities in the document

## ColPali: Visual Retriever based on PaliGemma-3B with ColBERT strategy


In [12]:
! pip install git+https://github.com/illuin-tech/colpali

  Cloning https://github.com/illuin-tech/colpali to /private/var/folders/gr/tvd1_9gs2_768mhrh885vz200000gn/T/pip-req-build-dp115pw9
  Running command git clone --filter=blob:none --quiet https://github.com/illuin-tech/colpali /private/var/folders/gr/tvd1_9gs2_768mhrh885vz200000gn/T/pip-req-build-dp115pw9
  Resolved https://github.com/illuin-tech/colpali to commit 8b01824546c62e46383ce26b439d9bfc6468f763
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached torch-2.2.2-cp39-none-macosx_10_9_x86_64.whl.metadata (25 kB)
  Using cached transformers-4.43.3-py3-none-any.whl.metadata (43 kB)
  Using cached GPUtil-1.4.0.tar.gz (5.5 kB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 7.4 MB/s eta 0:00:00
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ━━━━━━━━━━━━━━━━━━

In [21]:
! pip3 install --pre torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/nightly/cpu

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/nightly/cpu
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 51.5 MB/s eta 0:00:00


In [22]:
import torch
import math
# this ensures that the current MacOS version is at least 12.3+
print(torch.backends.mps.is_available())
# this ensures that the current current PyTorch installation was built with MPS activated.
print(torch.backends.mps.is_built())

True
True


In [14]:
import torch
import typer
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import AutoProcessor
from PIL import Image

from colpali_engine.models.paligemma_colbert_architecture import ColPali
from colpali_engine.trainer.retrieval_evaluator import CustomEvaluator
from colpali_engine.utils.colpali_processing_utils import process_images, process_queries
from colpali_engine.utils.image_from_page_utils import load_from_dataset

/Users/jvl/.pyenv/versions/3.9.17/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
# Load model
model_name = "vidore/colpali"
model = ColPali.from_pretrained("google/paligemma-3b-mix-448", torch_dtype=torch.bfloat16, device_map="cpu").eval()
model.load_adapter(model_name)
processor = AutoProcessor.from_pretrained(model_name)

Loading checkpoint shards: 100%|██████████| 3/3 [00:11<00:00,  3.75s/it]
Some weights of ColPali were not initialized from the model checkpoint at google/paligemma-3b-mix-448 and are newly initialized: ['custom_text_proj.bias', 'custom_text_proj.weight', 'language_model.lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
# select images -> load_from_pdf(<pdf_path>),  load_from_image_urls(["<url_1>"]), load_from_dataset(<path>)
#images = load_from_dataset("vidore/docvqa_test_subsampled")

import os
from PIL import Image
IMAGES_DIR = '/Users/jvl/Downloads/0000/images'
#load images with PIL

#0000177_page_1
images = [Image.open(os.path.join(IMAGES_DIR, i)) for i in os.listdir(IMAGES_DIR)[:100]]
print(f'Obtained {len(images)} images')

visual_entities = [
    'signature',
    'checkbox',
    'handwriting',
    'logo',
    'circled statement',
    'stamp',
    'barcode',
    'QRcode'
]

visual_entities_verbose = {
    'signature': {
        'description':
        'A handwritten depiction of someone\'s name, stylized and often unique to the individual. Used to signify agreement or authorization.',
        'common_locations':
        'At the bottom of letters, forms, contracts, and legal documents.',
        'characteristics': [
            'Stylized, often cursive writing',
            'Varies greatly between individuals',
            'May include embellishments or flourishes'
        ]
    },
    'checkbox': {
        'description':
        'A small box that can be checked or unchecked, used to indicate a choice or selection in forms and surveys.',
        'common_locations':
        'Forms, questionnaires, surveys, and checklists.',
        'characteristics': [
            'Square shape', 'Contains a checkmark or cross when selected',
            'Often accompanied by labels or instructions'
        ]
    },
    'handwriting': {
        'description':
        'Text written by hand, as opposed to printed text. Varies greatly between individuals and can convey personal style.',
        'common_locations':
        'Personal notes, annotations, handwritten letters, and signatures.',
        'characteristics': [
            'Varies in style, size, and slant',
            'May include cursive or printed letters',
            'Often less uniform than printed text'
        ]
    },
    'logo': {
        'description':
        'A graphic mark or symbol used to identify a company, organization, or brand. Logos are designed to be easily recognizable and memorable.',
        'common_locations':
        'Business cards, letterheads, advertisements, websites, and product packaging.',
        'characteristics': [
            'Distinctive shapes and colors',
            'May include text, images, or abstract designs',
            'Designed for brand recognition'
        ]
    },
    'circled statement': {
        'description':
        'A statement or piece of text that is circled to draw attention to it. Used for emphasis or to indicate importance.',
        'common_locations':
        'Documents with annotations, notes, and forms.',
        'characteristics': [
            'Text surrounded by a circle or oval',
            'Hand-drawn or printed circles', 'Highlights important information'
        ]
    },
    'stamp': {
        'description':
        'An imprint or mark, often made with ink, used to indicate approval, authentication, or status. Common in official documents.',
        'common_locations':
        'Official documents, certificates, letters, and packages.',
        'characteristics': [
            'Typically made with ink', 'May include text, dates, and images',
            'Often indicates approval or verification'
        ]
    },
    'barcode': {
        'description':
        'A machine-readable code consisting of parallel lines and spaces of varying widths. Used for identifying and tracking items.',
        'common_locations':
        'Product packaging, inventory labels, and tickets.',
        'characteristics': [
            'Series of parallel lines and spaces',
            'Varies in width and spacing',
            'Encodes information such as product numbers'
        ]
    },
    'QRcode': {
        'description':
        'A type of matrix barcode (or two-dimensional barcode) that contains information readable by a QR scanner or smartphone.',
        'common_locations':
        'Advertising materials, product packaging, business cards, and tickets.',
        'characteristics': [
            'Square grid of black and white squares',
            'Can encode URLs, contact information, and more',
            'Readable by cameras and QR code scanners'
        ]
    }
}
queries = [f"The page contains one or more of {entity} type" for entity in visual_entities]
# run inference - docs
dataloader = DataLoader(
    images,
    batch_size=4,
    shuffle=False,
    collate_fn=lambda x: process_images(processor, x),
)

Obtained 100 images


In [25]:

ds = []
for batch_doc in tqdm(dataloader):
    with torch.no_grad():
        batch_doc = {k: v.to(model.device) for k, v in batch_doc.items()}
        embeddings_doc = model(**batch_doc)
    ds.extend(list(torch.unbind(embeddings_doc.to("cpu"))))

  8%|▊         | 2/25 [3:03:09<35:06:16, 5494.63s/it]


KeyboardInterrupt: 

In [ ]:
# run inference - queries
dataloader = DataLoader(
    queries,
    batch_size=4,
    shuffle=False,
    collate_fn=lambda x: process_queries(processor, x, Image.new("RGB", (448, 448), (255, 255, 255))),
)

qs = []
for batch_query in dataloader:
    with torch.no_grad():
        batch_query = {k: v.to(model.device) for k, v in batch_query.items()}
        embeddings_query = model(**batch_query)
    qs.extend(list(torch.unbind(embeddings_query.to("cpu"))))

In [ ]:
retriever_evaluator = CustomEvaluator(is_multi_vector=True)
scores = retriever_evaluator.evaluate(qs, ds)
print(scores.argmax(axis=1))

#from this should be able to retrieve which pages most match the query